In [43]:
import requests
from bs4 import BeautifulSoup
import json
import re
import csv
import pandas as pd

In [44]:

url = "https://cymitquimica.com/categories/1828/nicotine-and-nicotine-derivatives/?srsltid=AfmBOor5CHkEY17td7i8alPNqfsjPX-VKsd6igxeoJFVzukYf576WD9_&page="
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

json_ld = None
for script in soup.find_all('script', type='application/ld+json'):
    try:
        json_ld = json.loads(script.string)
        break
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        continue

if json_ld:
    if isinstance(json_ld, dict):
        print("json_ld is a dictionary")
        print(json.dumps(json_ld, indent=2))
    else:
        print(f"Unexpected JSON-LD structure: {type(json_ld)}")
else:
    print("No valid JSON-LD found")


json_ld is a dictionary
{
  "@context": "http://schema.org",
  "@type": "WebSite",
  "url": "https://cymitquimica.com",
  "dateModified": "2019-02-15T00:00",
  "image": "https://static.cymitquimica.com/public/img/logo-cymit.png",
  "potentialAction": {
    "@type": "SearchAction",
    "target": "https://cymitquimica.com/search/{search_term_string}/",
    "query-input": "required name=search_term_string"
  },
  "sameAs": [
    "https://www.facebook.com/cymitquimica/",
    "https://twitter.com/cymitquimica",
    "https://es.linkedin.com/company/cymit-quimica-s-l-",
    "https://www.instagram.com/cymitquimica/",
    "https://www.pinterest.es/cymit/",
    "https://cymit.tumblr.com/"
  ]
}


In [45]:
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

product_links = soup.find_all("a", class_="js-product-link")

for link in product_links:
    product_url = link.get("href")
    if product_url:
        print(f"https://example.com{product_url}")


In [46]:
all_cas_numbers = []

for page in range(1, 14):
    urln = url + str(page)
    response = requests.get(urln)
    soup = BeautifulSoup(response.content, 'html.parser')

    for a_tag in soup.find_all('a', href=True):
        if "/cas/" in a_tag['href']:
            cas_number = a_tag.text.strip()
            if cas_number != "CAS list": print(cas_number)
            all_cas_numbers.append(cas_number)

df = pd.DataFrame(all_cas_numbers, columns = ["cas"])
unique_cas = df["cas"].drop_duplicates()
unique_cas_cleaned = unique_cas[~unique_cas.str.contains(r'[a-zA-Z]', regex=True)]

unique_cas_df = pd.DataFrame(unique_cas_cleaned, columns=["cas"])

total_original = len(df)
duplicates_removed = total_original - len(unique_cas)
invalid_rows_removed = len(unique_cas) - len(unique_cas_cleaned)

print(f"Total CAS Numbers: {total_original}")
print(f"Duplicates Removed: {duplicates_removed}")
print(f"Invalid Rows Removed (with letters): {invalid_rows_removed}")
print(f"Final Unique CAS Numbers: {len(unique_cas_cleaned)}")

494-97-3
485-35-8
95091-91-1
6456-44-6
532-12-7
20260-53-1
2743-90-0
609-71-2
59288-43-6
5470-70-2
3562-11-6
177785-14-7
29681-45-6
6960-22-1
1802-30-8
1215721-40-6
132334-98-6
89690-09-5
494-52-0
98-92-0
494-97-3
485-35-8
95091-91-1
6456-44-6
532-12-7
20260-53-1
2743-90-0
609-71-2
59288-43-6
5470-70-2
3562-11-6
177785-14-7
29681-45-6
6960-22-1
1802-30-8
1215721-40-6
132334-98-6
89690-09-5
494-52-0
98-92-0
6197-39-3
5398-44-7
61445-55-4
5006-66-6
5746-86-1
38496-18-3
39178-35-3
129747-52-0
38806-38-1
64091-91-4
486-56-6
1094-61-7
27828-71-3
78348-28-4
66093-90-1
1364663-27-3
13190-97-1
3222-56-8
1986-81-8
90872-72-3
6197-39-3
5398-44-7
61445-55-4
5006-66-6
5746-86-1
38496-18-3
39178-35-3
129747-52-0
38806-38-1
64091-91-4
486-56-6
1094-61-7
27828-71-3
78348-28-4
66093-90-1
1364663-27-3
13190-97-1
3222-56-8
1986-81-8
90872-72-3
59578-62-0
2047-49-6
6311-35-9
499-81-0
7076-23-5
614-00-6
2004-06-0
27247-34-3
3569-99-1
3222-47-7
321-02-8
13078-04-1
1314217-69-0
4314-66-3
5176-27-2
924-16-3


In [47]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
from tqdm import tqdm
import time
from selenium.webdriver.chrome.options import Options
import pubchempy as pcp

options = Options()
options.add_argument("--headless")  # Run in headless mode for speed
options.add_argument("--disable-gpu")  # Avoid GPU rendering issues
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)

# Set a page load timeout (e.g., 180 seconds)
driver.set_page_load_timeout(180)

In [48]:
def get_smiles_from_cas(cas_number):
    print(cas_number)
    try:
        compound = pcp.get_compounds(f'{cas_number}', 'name')[0]
        smiles = compound.isomeric_smiles
        print(f"SMILES for {cas_number}: {smiles}")
        return smiles
    except IndexError:
        print(f"No compound found for CAS: {cas_number}")
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [49]:
df = unique_cas_df

tqdm.pandas()
df['SMILES'] = df["cas"].progress_apply(get_smiles_from_cas)

total = len(df)
converted = df['SMILES'].notna().sum()
print(f"Total compounds: {total}")
print(f"Successfully converted: {converted}")
print(f"Conversion rate: {converted/total:.2%}")

df = df.dropna(subset=['SMILES'])
print(df)

df.to_csv('Unique_nicotine_derivatives_with_smiles.csv', index=False)

  0%|          | 0/172 [00:00<?, ?it/s]

494-97-3


  1%|          | 2/172 [00:00<00:39,  4.34it/s]

SMILES for 494-97-3: C1C[C@H](NC1)C2=CN=CC=C2
485-35-8


  2%|▏         | 3/172 [00:00<00:49,  3.43it/s]

...

SMILES for 107971-06-2: CC1=[N+](C=C(C=C1)C(=O)N)C.[I-]
857146-29-3


100%|██████████| 172/172 [01:14<00:00,  2.22it/s]

SMILES for 857146-29-3: CC1=CC(=CN=C1C)CO
20971-79-3


100%|██████████| 172/172 [01:14<00:00,  2.30it/s]

SMILES for 20971-79-3: C1CC(=O)OC1C2=CN=CC=C2
Total compounds: 172
Successfully converted: 164
Conversion rate: 95.35%
             cas                                             SMILES
0       494-97-3                           C1C[C@H](NC1)C2=CN=CC=C2
1       485-35-8                 C1[C@H]2CNC[C@@H]1C3=CC=CC(=O)N3C2
2     95091-91-1                             CN(C(=O)C1=CN=CC=C1)OC
3      6456-44-6                       C[N+]1=CC=CC(=C1)C(=O)N.[I-]
4       532-12-7                              C1CC(=NC1)C2=CN=CC=C2
..           ...                                                ...
486  887406-85-1             COC1=CC=C(C=C1)CN2C(CCC2=O)C3=CN=CC=C3
487    4315-37-1  CN1C=CC=C1C2=CN=CC=C2.[C@@H]([C@H](C(=O)O)O)(C...
488  107971-06-2                    CC1=[N+](C=C(C=C1)C(=O)N)C.[I-]
492  857146-29-3                                  CC1=CC(=CN=C1C)CO
493   20971-79-3                             C1CC(=O)OC1C2=CN=CC=C2

[164 rows x 2 columns]


In [50]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as GeoDataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from gnn import GNNModel, smiles_to_graph, MoleculeDataset

In [51]:
df = pd.read_csv('Unique_nicotine_derivatives_with_smiles.csv')
df_cleaned = df[df['SMILES'].notna() & (df['SMILES'] != '')]
df_cleaned.to_csv('cleaned_nicotine_derivatives_with_smiles.csv', index=False)

In [ ]:
df = pd.read_csv('Unique_nicotine_derivatives_with_smiles.csv')
df_cleaned = df[df['SMILES'].notna() & (df['SMILES'] != '')]
df_cleaned.to_csv('cleaned_nicotine_derivatives_with_smiles.csv', index=False)

In [52]:
df = pd.read_csv('cleaned_nicotine_derivatives_with_smiles.csv')
nicotine_smiles = df['SMILES']

nicotine_graphs = [graph for graph in [smiles_to_graph(smile) for smile in nicotine_smiles] if graph is not None]

model_path = 'gnn_model.pth'
model = GNNModel(num_node_features=26, hidden_dim=128, output_dim=2)
model.load_state_dict(torch.load(model_path))
model.eval()

predictions = []

with torch.no_grad():
    for graph in nicotine_graphs:
        output = model(graph)
        _, predicted = torch.max(output, dim=1)
        predictions.append(predicted.item())

df['Predicted Label'] = predictions

df.to_csv('nicotine_derivatives_with_predictions.csv', index=False)

total = len(df)
predicted_count = len(predictions)
print(f"Total rows: {total}")
print(f"Predictions made: {predicted_count}")
print(f"Updated CSV saved with predictions.")

Total rows: 164
Predictions made: 164
Updated CSV saved with predictions.
